 ### 1. 서울시 공공데이터 제공 장소 116곳 리스트 뽑기


In [ ]:
##### API 가져와보기
import requests
import pandas as pd 
import json
from urllib.parse import quote
from bs4 import BeautifulSoup

# 서울 지역 116곳 번호 뽑기
area_list = []
try:
    for i in range(,63):
        area = f"POI{i:03d}"

        # API 키
        api_key = '6e51654f776b353335387168745664'
        # API URL 구성
        url = f'http://openapi.seoul.go.kr:8088/{api_key}/xml/citydata/1/1000/{area}'
        # API 요청, 응답
        response = requests.get(url)
        xml_data = response.text
        # xml파싱
        soup = BeautifulSoup(xml_data, 'lxml-xml')
        area_name = soup.find('AREA_NM').text 
        address = soup.find('ROAD_ADDR')
        print(soup)
        print(area_name)

except :
    # 오류 발생 시 실행되는 코드
    print("오류가 발생했습니다.")

<?xml version="1.0" encoding="utf-8"?>
<SeoulRtd.citydata><list_total_count>1</list_total_count><RESULT><RESULT.CODE>INFO-000</RESULT.CODE><RESULT.MESSAGE>정상 처리되었습니다.</RESULT.MESSAGE></RESULT><CITYDATA><AREA_NM>낙산공원·이화마을</AREA_NM><AREA_CD>POI062</AREA_CD><LIVE_PPLTN_STTS><LIVE_PPLTN_STTS><AREA_NM>낙산공원·이화마을</AREA_NM><AREA_CD>POI062</AREA_CD><AREA_CONGEST_LVL>여유</AREA_CONGEST_LVL><AREA_CONGEST_MSG>사람이 몰려있을 가능성이 낮고 붐빔은 거의 느껴지지 않아요. 도보 이동이 자유로워요.</AREA_CONGEST_MSG><AREA_PPLTN_MIN>100</AREA_PPLTN_MIN><AREA_PPLTN_MAX>200</AREA_PPLTN_MAX><MALE_PPLTN_RATE>55.0</MALE_PPLTN_RATE><FEMALE_PPLTN_RATE>45.0</FEMALE_PPLTN_RATE><PPLTN_RATE_0>0.0</PPLTN_RATE_0><PPLTN_RATE_10>4.5</PPLTN_RATE_10><PPLTN_RATE_20>18.5</PPLTN_RATE_20><PPLTN_RATE_30>14.4</PPLTN_RATE_30><PPLTN_RATE_40>13.7</PPLTN_RATE_40><PPLTN_RATE_50>24.7</PPLTN_RATE_50><PPLTN_RATE_60>12.7</PPLTN_RATE_60><PPLTN_RATE_70>11.5</PPLTN_RATE_70><RESNT_PPLTN_RATE>31.2</RESNT_PPLTN_RATE><NON_RESNT_PPLTN_RATE>68.8</NON_RESNT_PPLTN_RATE><REPLACE_YN>N</

### 2. 가져온 지역 한개에서 요소 가져오기 

In [ ]:
# 특정 요소 내의 모든 항목 찾기
live_ppltn = soup.find('LIVE_PPLTN_STTS')
#지역
area_nm = live_ppltn.find('AREA_NM').text
#혼잡도
congestion = live_ppltn.find('AREA_CONGEST_LVL').text
print(f"지역: {area_nm}, 혼잡도: {congestion}")
#자치구 
address = soup.find('ADDRESS').text
print(address)


None


#### 2.1데이터분석에 필요한 값들이 잘나오는지 하나씩 확인해보자

In [ ]:

## 서울시 실시간 데이터 가져오기

#데이트 날씨 점수 = 강수점수(30) + 기온점수(20) + 대기질점수(15) + 하늘상태점수(15) 

#### 가져올 값
#핫스팟 장소명(LIVE_PPLTN_STTS), 

#### 날씨 정보
#기온(TEMP), 강수(PRECIPITATION),강수확률(RAIN_CHANCE), 강수형태(PRECPT_TYPE), 대기질(AIR_IDX), 하늘상태(SKY_STTS), 습도(HUMIDITY), 자외선(UV_INDEX)

#### 혼잡도 정보
#장소 혼잡도 지표(AREA_CONGEST_LVL), 실시간 상권현황 (LIVE_CMRCL_STTS),  장소 예측 혼잡도 지표 (FCST_CONGEST_LVL), 장소 실시간 상권 현황(AREA_CMRCL_LVL)

##불쾌지수 = 1.8 x 기온 – 0.55 x (1 – 습도) x (1.8 x 기온 – 26) + 32

In [12]:
rain_mm =[]
# 강수량 (-일때 0으로 변경)
rain_mm_list = soup.find_all('PRECIPITATION')
for rain in rain_mm_list:
    rain = rain.text
    if rain == "-":
        rain_mm.append("0")
    else:
        rain_mm.append(rain)
    print(rain)
    
print(len(rain_mm))

-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
25


In [ ]:
#온도
temp = soup.find_all('TEMP')
print(temp)
print(len(temp))

[<TEMP>15.2</TEMP>, <TEMP>14</TEMP>, <TEMP>13</TEMP>, <TEMP>12</TEMP>, <TEMP>10</TEMP>, <TEMP>10</TEMP>, <TEMP>9</TEMP>, <TEMP>9</TEMP>, <TEMP>9</TEMP>, <TEMP>9</TEMP>, <TEMP>9</TEMP>, <TEMP>8</TEMP>, <TEMP>8</TEMP>, <TEMP>8</TEMP>, <TEMP>9</TEMP>, <TEMP>10</TEMP>, <TEMP>12</TEMP>, <TEMP>14</TEMP>, <TEMP>17</TEMP>, <TEMP>19</TEMP>, <TEMP>20</TEMP>, <TEMP>17</TEMP>, <TEMP>16</TEMP>, <TEMP>14</TEMP>, <TEMP>13</TEMP>]
25


In [ ]:
#강수확률
rain_pct = soup.find_all('RAIN_CHANCE') 
print(rain_pct)
print(len(rain_pct))

[<RAIN_CHANCE>0</RAIN_CHANCE>, <RAIN_CHANCE>0</RAIN_CHANCE>, <RAIN_CHANCE>0</RAIN_CHANCE>, <RAIN_CHANCE>0</RAIN_CHANCE>, <RAIN_CHANCE>0</RAIN_CHANCE>, <RAIN_CHANCE>0</RAIN_CHANCE>, <RAIN_CHANCE>0</RAIN_CHANCE>, <RAIN_CHANCE>20</RAIN_CHANCE>, <RAIN_CHANCE>30</RAIN_CHANCE>, <RAIN_CHANCE>20</RAIN_CHANCE>, <RAIN_CHANCE>0</RAIN_CHANCE>, <RAIN_CHANCE>0</RAIN_CHANCE>, <RAIN_CHANCE>30</RAIN_CHANCE>, <RAIN_CHANCE>30</RAIN_CHANCE>, <RAIN_CHANCE>20</RAIN_CHANCE>, <RAIN_CHANCE>20</RAIN_CHANCE>, <RAIN_CHANCE>30</RAIN_CHANCE>, <RAIN_CHANCE>20</RAIN_CHANCE>, <RAIN_CHANCE>20</RAIN_CHANCE>, <RAIN_CHANCE>30</RAIN_CHANCE>, <RAIN_CHANCE>20</RAIN_CHANCE>, <RAIN_CHANCE>0</RAIN_CHANCE>, <RAIN_CHANCE>0</RAIN_CHANCE>, <RAIN_CHANCE>20</RAIN_CHANCE>]
24


In [ ]:
#강수형태
rain_type = soup.find_all('PRECPT_TYPE') # 없음 출력
print(rain_type)
print(len(rain_type))

[<PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>]
25


In [ ]:
#대기질
air_idx = soup.find_all('AIR_IDX') #24동안 1하나만
print(air_idx)
print(len(air_idx))

[<AIR_IDX>나쁨</AIR_IDX>]
1


In [ ]:
#하늘상태
sty_stts = soup.find_all('SKY_STTS') #24동안 1하나만
print(sty_stts)
print(len(sty_stts))

[<SKY_STTS>구름많음</SKY_STTS>, <SKY_STTS>구름많음</SKY_STTS>, <SKY_STTS>맑음</SKY_STTS>, <SKY_STTS>맑음</SKY_STTS>, <SKY_STTS>맑음</SKY_STTS>, <SKY_STTS>맑음</SKY_STTS>, <SKY_STTS>맑음</SKY_STTS>, <SKY_STTS>구름많음</SKY_STTS>, <SKY_STTS>흐림</SKY_STTS>, <SKY_STTS>구름많음</SKY_STTS>, <SKY_STTS>맑음</SKY_STTS>, <SKY_STTS>맑음</SKY_STTS>, <SKY_STTS>흐림</SKY_STTS>, <SKY_STTS>흐림</SKY_STTS>, <SKY_STTS>구름많음</SKY_STTS>, <SKY_STTS>구름많음</SKY_STTS>, <SKY_STTS>흐림</SKY_STTS>, <SKY_STTS>구름많음</SKY_STTS>, <SKY_STTS>구름많음</SKY_STTS>, <SKY_STTS>흐림</SKY_STTS>, <SKY_STTS>구름많음</SKY_STTS>, <SKY_STTS>맑음</SKY_STTS>, <SKY_STTS>맑음</SKY_STTS>, <SKY_STTS>구름많음</SKY_STTS>]
24


In [ ]:
#습도
humidity = soup.find_all('HUMIDITY') 
print(humidity)
print(len(humidity))

[<HUMIDITY>64</HUMIDITY>]
1


#### 오후 온도 구하기

In [ ]:
from collections import Counter

fcst24hours = soup.find('FCST24HOURS')  # 단일 컨테이너 요소를 찾음
fcst_items = fcst24hours.find_all('FCST_DT')  # 또는 실제 항목 태그명으로 변경
temp_list=[]
sky_status_list = []
for item in fcst_items:
    hours = item.text
   
    # 원하는 시간대 확인 (12시~18시)
    if any(time in hours for time in ['1200', '1300', '1400', '1500', '1600', '1700', '1800']):

        
        parent = item.parent
        temp = int(parent.find('TEMP').text)
        temp_list.append(temp)
        sky_status_list.append(parent.find('SKY_STTS').text) #24동안 1하나만
        print(sty_stts)
# 오후 하늘상태 구하기
counter = Counter(sky_status_list)

# 온도 평균 구하기(합/개수)
avg_temp = int(sum(temp_list) / len(temp_list))
print(avg_temp)

구름많음
구름많음
구름많음
흐림
구름많음
맑음
맑음
16


In [137]:
# 반복해서 구해야할 온도계수


### 오후날씨 함수만들기

In [47]:
#리스트 가져오기
def get_after_wether(value):
    from collections import Counter

    fcst24hours = soup.find('FCST24HOURS')  # 단일 컨테이너 요소를 찾음
    fcst_items = fcst24hours.find_all('FCST_DT')  # 또는 실제 항목 태그명으로 변경

    after_list =[]
    #리스트마다 행하나씩 뽑기
    for item in fcst_items:
        hours = item.text
        
        if any(time in hours for time in ['1100','1200', '1300', '1400', '1500', '1600', '1700', '1800']):
            #상위 요소로 올라가기
            parent = item.parent
            
            #value가 온도일때 온도값 주기
            if value == 'temp':
                try:
                    temp = int(parent.find('TEMP').text)
                    after_list.append(temp)
                except:
                    print("온도값에 오류가 발생")
            #하늘상태일때 하늘상태값 주기 # 맑음 문자로 나옴 카운드 ?
            elif value == 'sky_stts':
                try:
                    sky_stts = parent.find('SKY_STTS').text
                    after_list.append(sky_stts)
                    print(sky_stts)
                except:
                    print("하늘상태값에 오류가 발생")
            #강수확률일때 강수확률값주기
            elif value == 'rain_chance':
                try:
                    rain_chance = int(parent.find('RAIN_CHANCE').text)
                    
                    after_list.append(rain_chance)
                    print(rain_chance)
                except:
                    print("강수확률값에 오류가 발생")

            # 강수량일때 (-일때 0으로 변경)
            elif value == 'rain_mm':
                try:
                    rain = parent.find('PRECIPITATION').text
                    if rain == "-":
                        after_list.append(int("0"))
                    else:
                        after_list.append(int(rain))
                    print(rain)
                except:
                    #오류발생
                    print("강수량 값에 대한 오류가 발생")   

    # 리스트 평균 구하기(합/개수)
    if value in ['temp', 'rain_chance', 'rain_mm']:
        avg_after = int(sum(after_list) / len(after_list))
        return avg_after
    else:
        return Counter(after_list).most_common(1)[0][0]

           

### 인구혼잡도 함수

In [83]:

        
def get_many_people():
    from collections import Counter
    fcst24hours = soup.find('FCST_PPLTN')  # 단일 컨테이너 요소를 찾음
    fcst_items = fcst24hours.find_all('FCST_TIME')  # 또는 실제 항목 태그명으로 변경
    after_list =[]
    #리스트마다 행하나씩 뽑기
    for item in fcst_items:
        print(item)
        hours = item.text
        print(hours)
        if any(time in hours for time in ['11:00','12:00', '13:00', '14:00', '15:00', '16:00', '17:00', '18:00']):
            try:
                parent= item.parent
                many_level=parent.find('FCST_CONGEST_LVL').text
                after_list.append(many_level)
                print(many_level)
            except:
                print("혼잡도값에 대한 오류")
    
    return Counter(after_list).most_common(1)[0][0]


In [50]:
aaa = get_after_wether('sky_stts')
print("값 :"+ aaa)

흐림
구름많음
구름많음
흐림
구름많음
맑음
맑음
구름많음
값 :구름많음


In [64]:
bbb = get_many_people()
print(bbb)

IndexError: list index out of range

In [ ]:
#자외선
uv_index = soup.find_all('UV_INDEX') 
print(uv_index)
print(len(uv_index))

[<UV_INDEX>낮음</UV_INDEX>]
1


#### 2.2 실시간 인구 지표값 가져오기

In [ ]:
area_name = soup.find_all('AREA_NM') 
parent = area_name.parent
area_person_many = soup.find_all('AREA_CONGEST_LVL') 
live_cmrcl_stts = soup.find_all('LIVE_CMRCL_STTS') 
fcst_person_many = soup.find_all('FCST_CONGEST_LVL') # 12시간 예측
area_cmrcl_lvl = soup.find_all('AREA_CMRCL_LVL') 

fcst_person_many = soup.find_all('FCST_CONGEST_LVL') # 12시간 예측



#핫스팟장소명
print(area_name)
print(len(area_name))
# 장소 혼잡도 지표
print(area_person_many)
print(len(area_person_many))
# 실시간 상권현황
print(live_cmrcl_stts)
print(len(live_cmrcl_stts))
# 장소 예측 혼잡도 지표 
print(fcst_person_many)
print(len(fcst_person_many))
# 장소 실시간 상권 현황
print(area_cmrcl_lvl)
print(len(area_cmrcl_lvl))

[<AREA_NM>노들섬</AREA_NM>, <AREA_NM>노들섬</AREA_NM>]
2
[<AREA_CONGEST_LVL>여유</AREA_CONGEST_LVL>]
1
[<LIVE_CMRCL_STTS/>]
1
[<FCST_CONGEST_LVL>보통</FCST_CONGEST_LVL>, <FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL>, <FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL>, <FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL>, <FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL>, <FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL>, <FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL>, <FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL>, <FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL>, <FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL>, <FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL>, <FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL>]
12
[]
0


In [48]:
fcst_person_many = soup.find_all('FCST_TIME') # 12시간 예측

for i in fcst_person_many:
    parent = i.parent
    print(parent)
    print(i)


print(len(fcst_person_many))

<FCST_PPLTN><FCST_TIME>2025-03-25 00:00</FCST_TIME><FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL><FCST_PPLTN_MIN>7500</FCST_PPLTN_MIN><FCST_PPLTN_MAX>8000</FCST_PPLTN_MAX></FCST_PPLTN>
<FCST_TIME>2025-03-25 00:00</FCST_TIME>
<FCST_PPLTN><FCST_TIME>2025-03-25 01:00</FCST_TIME><FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL><FCST_PPLTN_MIN>6000</FCST_PPLTN_MIN><FCST_PPLTN_MAX>6500</FCST_PPLTN_MAX></FCST_PPLTN>
<FCST_TIME>2025-03-25 01:00</FCST_TIME>
<FCST_PPLTN><FCST_TIME>2025-03-25 02:00</FCST_TIME><FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL><FCST_PPLTN_MIN>5000</FCST_PPLTN_MIN><FCST_PPLTN_MAX>5500</FCST_PPLTN_MAX></FCST_PPLTN>
<FCST_TIME>2025-03-25 02:00</FCST_TIME>
<FCST_PPLTN><FCST_TIME>2025-03-25 03:00</FCST_TIME><FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL><FCST_PPLTN_MIN>4500</FCST_PPLTN_MIN><FCST_PPLTN_MAX>5000</FCST_PPLTN_MAX></FCST_PPLTN>
<FCST_TIME>2025-03-25 03:00</FCST_TIME>
<FCST_PPLTN><FCST_TIME>2025-03-25 04:00</FCST_TIME><FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL><FCST_PPLTN_MIN>4500</FCST_PPLTN_MIN><

### 3.합쳐보기

구해야할 값
구, 명소명, 주소, 날씨점수, 혼잡도 점수,

In [3]:
##### API 가져와보기
import requests
import pandas as pd 
import json
from urllib.parse import quote
from bs4 import BeautifulSoup

# 서울 지역 116곳 번호 뽑기
total_list = []

for i in range(1,117):
    area_list = []
    #지역 코드 값 자동 완성
    area = f"POI{i:03d}"
    # API 키
    api_key = '6e51654f776b353335387168745664'
    # API URL 구성
    url = f'http://openapi.seoul.go.kr:8088/{api_key}/xml/citydata/1/2/{area}'
    # API 요청, 응답
    response = requests.get(url)
    xml_data = response.text
    # xml파싱
    soup = BeautifulSoup(xml_data, 'lxml-xml')


    
    # 특정 요소 내의 모든 항목 찾기
    live_ppltn = soup.find('LIVE_PPLTN_STTS')
    #지역
    area_nm = live_ppltn.find('AREA_NM').text
    area_list.append(area_nm)
    #혼잡도
    try:
        #주소 
        address = soup.find('ADDRESS').text
        area_list.append(address)
        #구
        gu = address.split(" ")[0]
        area_list.append(gu)
        #날씨점수
    except:
        area_list.append("주소가 안찾아짐")
    #혼잡도
    total_list.append(area_list)
    print(area_list)
   

['강남 MICE 관광특구', '강남구 삼성동 159-0', '강남구']
['동대문 관광특구', '중구 을지로7가 2-36', '중구']
['명동 관광특구', '중구 소공동 87-3 0', '중구']
['이태원 관광특구', '용산구 이태원동 127-36', '용산구']
['잠실 관광특구', '송파구 잠실동 40-1', '송파구']
['종로·청계 관광특구', '종로구 종로5가 398-3', '종로구']
['홍대 관광특구', '마포구 동교동 162-4', '마포구']
['경복궁', '종로구 내자동 35-0 내자동 54(치과의원) ~ 필운동 227(금고 앞)', '종로구']
['광화문·덕수궁', '중구 정동 5-5 (정동 5-5(시청역 2번 출구 앞)', '중구']
['보신각', '종로구 청진동 70-0', '종로구']
['서울 암사동 유적', '강동구 암사동 139-2', '강동구']
['창덕궁·종묘', '종로구 와룡동 206-0', '종로구']
['가산디지털단지역', '금천구 가산동 371-28', '금천구']
['강남역', '서초구 서초동 1330-0', '서초구']
['건대입구역', '광진구 자양동 5-5', '광진구']
['고덕역', '강동구 명일동 48-3', '강동구']
['고속터미널역', '서초구 반포동 118-3', '서초구']
['교대역', '서초구 서초동 1674-4', '서초구']
['구로디지털단지역', '구로구 구로동 1124-49', '구로구']
['구로역', '구로구 구로동 586-5', '구로구']
['군자역', '광진구 중곡동 649-4', '광진구']
['남구로역', '구로구 구로동 806-73', '구로구']
['대림역', '영등포구 대림동 808-24', '영등포구']
['동대문역', '종로구 종로6가 289-49', '종로구']
['뚝섬역', '성동구 성수동1가 656-75', '성동구']
['미아사거리역', '강북구 미아동 60-11', '강북구']
['발산역', '강서구 마곡동 797-0', '강서구']
['북한산우이역', 

In [ ]:
##### API 가져와보기
import requests
import pandas as pd 
import json
from urllib.parse import quote
from bs4 import BeautifulSoup

# 서울 지역 116곳 번호 뽑기
area_list = []
try:
    area = 'POI061'
    # API 키
    api_key = '78464779476b353336336774526353'
    # API URL 구성
    url = f'http://openapi.seoul.go.kr:8088/{api_key}/xml/citydata/1/100/{area}'
    # API 요청, 응답
    response = requests.get(url)
    xml_data = response.text
    # xml파싱
    soup = BeautifulSoup(xml_data, '-xml')
    print(soup)
    address = soup.find('ADDRESS')
    print(address)

except :
    # 오류 발생 시 실행되는 코드
    print("오류가 발생했습니다.")

오류가 발생했습니다.


In [4]:
print(len(total_list))

116


### 도시데이터 인구밀집지역이 상권이 발달되어 데이트 지역으로 적합한지 
### 아니면 단순 출퇴근으로 인한 인구 밀집 지표가 높은건지 확인 

In [7]:
import pandas as pd
df = pd.DataFrame(total_list, columns=['지역', '주소','구'])

In [12]:
df.head(20)

,지역,주소,구
0,강남 MICE 관광특구,강남구 삼성동 159-0,강남구
1,동대문 관광특구,중구 을지로7가 2-36,중구
2,명동 관광특구,중구 소공동 87-3 0,중구
3,이태원 관광특구,용산구 이태원동 127-36,용산구
4,잠실 관광특구,송파구 잠실동 40-1,송파구
5,종로·청계 관광특구,종로구 종로5가 398-3,종로구
6,홍대 관광특구,마포구 동교동 162-4,마포구
7,경복궁,종로구 내자동 35-0 내자동 54(치과의원) ~ 필운동 227(금고 앞),종로구
8,광화문·덕수궁,중구 정동 5-5 (정동 5-5(시청역 2번 출구 앞),중구
9,보신각,종로구 청진동 70-0,종로구


In [50]:
import re
filtered = df[df['지역'].str.contains('역')]['지역']
sub_filter = [re.split('·', station)[0] for station in filtered]

print(sub_filter)


['가산디지털단지역', '강남역', '건대입구역', '고덕역', '고속터미널역', '교대역', '구로디지털단지역', '구로역', '군자역', '남구로역', '대림역', '동대문역', '뚝섬역', '미아사거리역', '발산역', '북한산우이역', '사당역', '삼각지역', '서울대입구역', '서울식물원', '서울역', '선릉역', '성신여대입구역', '수유역', '신논현역', '신도림역', '신림역', '신촌', '양재역', '역삼역', '연신내역', '오목교역', '왕십리역', '용산역', '이태원역', '장지역', '장한평역', '천호역', '총신대입구(이수)역', '충정로역', '합정역', '혜화역', '홍대입구역(2호선)', '회기역', '방배역 먹자골목']


In [51]:
print(sub_filter)

['가산디지털단지역', '강남역', '건대입구역', '고덕역', '고속터미널역', '교대역', '구로디지털단지역', '구로역', '군자역', '남구로역', '대림역', '동대문역', '뚝섬역', '미아사거리역', '발산역', '북한산우이역', '사당역', '삼각지역', '서울대입구역', '서울식물원', '서울역', '선릉역', '성신여대입구역', '수유역', '신논현역', '신도림역', '신림역', '신촌', '양재역', '역삼역', '연신내역', '오목교역', '왕십리역', '용산역', '이태원역', '장지역', '장한평역', '천호역', '총신대입구(이수)역', '충정로역', '합정역', '혜화역', '홍대입구역(2호선)', '회기역', '방배역 먹자골목']


In [101]:
import pandas as pd
subway_level1231 = pd.read_csv("/home/mac/project_ws/서울교통공사_지하철혼잡도정보_20241231.csv",encoding="euc-kr", index_col = 0)
subway_level1231

,요일구분,호선,역번호,출발역,상하구분,5시30분,6시00분,6시30분,7시00분,7시30분,...,20시00분,20시30분,21시00분,21시30분,22시00분,22시30분,23시00분,23시30분,00시00분,00시30분
연번,,,,,,,,,,,,,,,,,,,,,
1,평일,1,150,서울역,상선,8.6,20.9,20.4,33.1,61.1,...,19.3,18.6,19.1,15.4,16.7,18.1,13.0,12.4,6.5,1.0
2,평일,1,151,시청,상선,8.6,15.6,17.0,26.8,46.4,...,23.2,23.0,23.8,18.6,20.9,20.7,17.7,14.0,7.0,1.1
3,평일,1,152,종각,상선,8.4,12.5,10.8,18.1,27.3,...,30.1,29.7,32.6,24.7,27.4,26.7,24.4,16.2,7.8,1.1
4,평일,1,153,종로3가,상선,7.2,13.8,10.9,18.7,27.6,...,30.6,32.0,38.7,27.4,30.6,30.4,27.3,18.3,8.0,1.1
5,평일,1,154,종로5가,상선,6.6,12.4,8.8,14.0,21.2,...,32.4,33.5,39.2,33.5,31.8,28.1,27.5,17.8,12.8,2.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1658,일요일,8,2826,수진,하선,9.6,4.0,4.6,4.0,4.2,...,2.7,1.9,2.2,1.8,1.6,1.3,0.8,0.6,0.1,0.0
1659,일요일,8,2827,모란,상선,1.4,1.0,2.2,1.5,1.6,...,5.9,6.2,6.2,5.9,4.5,4.5,3.3,4.2,0.0,0.0
1660,일요일,8,2827,모란,하선,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [102]:
# 리스트에서 '역' 글자 제거

filtered_clean = [name.replace('역', '') for name in sub_filter]

# DataFrame에서도 공백 제거 (안전하게)
subway_level1231['출발역'] = subway_level1231['출발역'].str.strip()

# 필터링
filtered_df = subway_level1231[subway_level1231['출발역'].isin(filtered_clean)]

In [103]:
filtered_df

,요일구분,호선,역번호,출발역,상하구분,5시30분,6시00분,6시30분,7시00분,7시30분,...,20시00분,20시30분,21시00분,21시30분,22시00분,22시30분,23시00분,23시30분,00시00분,00시30분
연번,,,,,,,,,,,,,,,,,,,,,
6,평일,1,155,동대문,상선,9.4,12.6,8.6,13.5,19.9,...,35.2,30.5,39.6,33.2,32.0,28.2,29.8,19.1,13.2,2.4
16,평일,1,155,동대문,하선,14.1,18.6,22.2,42.7,67.8,...,14.8,13.8,15.0,12.6,14.3,9.2,7.2,7.5,3.0,0.5
28,평일,2,208,왕십리,내선,19.3,30.5,24.6,30.2,45.0,...,18.2,17.3,22.6,25.8,24.3,27.7,17.0,21.8,8.8,5.8
30,평일,2,210,뚝섬,내선,18.3,25.6,21.2,26.4,33.9,...,21.4,17.3,22.7,30.7,25.2,29.3,19.0,21.4,10.4,4.5
32,평일,2,212,건대입구,내선,11.3,20.8,18.4,22.1,33.4,...,26.4,25.5,33.2,32.4,30.9,29.0,18.7,18.0,16.3,10.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1616,일요일,7,2748,가산디지털단지,하선,11.0,11.1,12.6,10.4,9.4,...,30.8,32.2,32.1,32.1,25.4,22.3,23.5,10.6,4.0,0.0
1629,일요일,8,2812,천호,상선,2.4,5.2,4.1,4.6,4.9,...,29.4,29.5,31.2,26.8,27.8,28.4,19.2,12.2,0.0,0.0
1630,일요일,8,2812,천호,하선,13.3,19.7,15.5,19.4,21.9,...,23.3,22.6,19.3,18.6,17.6,11.8,9.1,6.1,0.0,0.0


In [ ]:
#7시부터
weektime_start = filtered_df.columns[16:-9]  
print(weektime_start)

Index(['10시30분', '11시00분', '11시30분', '12시00분', '12시30분', '13시00분', '13시30분',
       '14시00분', '14시30분', '15시00분', '15시30분', '16시00분', '16시30분', '17시00분',
       '17시30분', '18시00분', '18시30분', '19시00분', '19시30분', '20시00분'],
      dtype='object')


In [123]:
# 0. 평일 데이터만 추출
weekday_df = filtered_df[filtered_df['요일구분'] == '평일'].copy()
weektime_start = weekday_df.columns[8:15]   
weektime_end = weekday_df.columns[-16:-7]    

# 1. 혼잡도 평균 계산 (평일)
weekday_df['출근_평균혼잡도'] = weekday_df[weektime_start].mean(axis=1).round().astype(int)
weekday_df['퇴근_평균혼잡도'] = weekday_df[weektime_end].mean(axis=1).round().astype(int)

# 2. 출발역별 최고 혼잡도 기준 행 추출 (평일)
max_idx_start = weekday_df.groupby('출발역')['출근_평균혼잡도'].idxmax()
max_idx_end = weekday_df.groupby('출발역')['퇴근_평균혼잡도'].idxmax()

# 3. 최종 평일 DataFrame
time_start_weekday = weekday_df.loc[max_idx_start].reset_index(drop=True)
time_end_weekday = weekday_df.loc[max_idx_end].reset_index(drop=True)

# 4. 주말 데이터 추출
weekend_df = filtered_df[filtered_df['요일구분'].isin(['토요일', '일요일'])].copy()

# 주말 혼잡도 평균 계산
weekend_df['주말_평균혼잡도'] = weekend_df[weektime_start].mean(axis=1).round().astype(int)

# 주말 가장 혼잡한 출발역 추출
max_idx_weekend = weekend_df.groupby('출발역')['주말_평균혼잡도'].idxmax()

# 최종 주말 DataFrame
time_weekend = weekend_df.loc[max_idx_weekend].reset_index(drop=True)

# 5. 원하는 열만 추출
weekday_summary = time_start_weekday[['출발역', '출근_평균혼잡도']].copy()
weekday_summary['퇴근_평균혼잡도'] = time_end_weekday.set_index('출발역')['퇴근_평균혼잡도']

# 출근, 퇴근, 주말 각각 추출
start = time_start_weekday[['출발역', '출근_평균혼잡도']]
end = time_end_weekday[['출발역', '퇴근_평균혼잡도']]
weekend = time_weekend[['출발역', '주말_평균혼잡도']]

# 병합
merged_df = (
    start
    .merge(end, on='출발역', how='left')
    .merge(weekend, on='출발역', how='left')
)

# 공백 제거 (병합 안정성 확보용)
for df in [time_start_weekday, time_end_weekday, time_weekend]:
    df['출발역'] = df['출발역'].str.strip()

# 병합 후 NaN 처리
merged_df['주말_평균혼잡도'] = merged_df['주말_평균혼잡도'].fillna(0).astype(int)
# 최종 출력
print(merged_df)

        출발역  출근_평균혼잡도  퇴근_평균혼잡도  주말_평균혼잡도
0   가산디지털단지        76        88         0
1        강남        84        94         0
2      건대입구        94        98         0
3        고덕        71        68         0
4     고속터미널        71        79         0
5        교대       102        99         0
6   구로디지털단지        55        60         0
7        군자       120        96         0
8       남구로        71        64         0
9        대림        58        61         0
10      동대문        97        91         0
11       뚝섬        35        50         0
12       발산        25        29         0
13       사당       116        94         0
14      삼각지        37        45         0
15    서울대입구        98        78         0
16       선릉        59        66         0
17   성신여대입구       102        81         0
18       수유        75        55         0
19      신도림        73        51         0
20       신림        79        59         0
21       양재        39        42         0
22      연신내        64        52   

In [ ]:
time_end